In [1]:
import gurobipy as gp
import pandas as pd
from gurobipy import GRB

In [2]:
costs_data_new = pd.read_csv("/Users/mahinbindra/Downloads/costs.csv")

In [3]:
num_sites = costs_data_new.shape[0]
total_units = 2900000

In [4]:
# Fixed and variable costs per site
fixed_costs = costs_data_new['Fixed'].tolist()
variable_costs = costs_data_new['Variable'].tolist()

In [5]:
from gurobipy import Model, GRB, quicksum

# Create a new model
model = Model("HealthLink Supplies Distribution")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-15


In [6]:
# Decision variables
# Binary variables for each site (1 if a warehouse is established, 0 otherwise)
x = model.addVars(27, vtype=GRB.BINARY, name="x")

# Continuous variables for the number of units transported from Ottawa to each site
y = model.addVars(27, lb=0, name="y")

In [7]:
# Objective function: Minimize the total cost
obj = quicksum(fixed_costs[i] * x[i] + variable_costs[i] * y[i] for i in range(27))
model.setObjective(obj, GRB.MINIMIZE)

In [8]:
model.addConstr(quicksum(y[i] for i in range(27)) <= 2900000, "supply_constraint")

# Constraint 2: Demand constraints (using a large number M)
M = 1000000  # Adjust M based on the context, it should be larger than the maximum possible demand
for i in range(27):
    model.addConstr(y[i] <= M * x[i], f"demand_constraint_{i+1}")
    
# Constraint 5: Storage capacity constraints
for i in range(27):
    model.addConstr(175000 * x[i] <= y[i], f"min_storage_{i+1}")
    model.addConstr(y[i] <= 375000 * x[i], f"max_storage_{i+1}")

# Constraint 6: Location choice constraints
# At least four locations must be chosen among sites 6-16
model.addConstr(quicksum(x[i] for i in range(5, 16)) >= 4, "min_locations_6_16")

# No more than 6 locations must be chosen among the even-numbered sites
model.addConstr(quicksum(x[i] for i in range(1, 27, 2)) <= 6, "max_even_locations")

# If location 1 or 2 is chosen, then sites 5, 6, and 7 cannot be chosen
model.addConstr(x[0] + x[1] <= 1 - x[4], "exclusion_1_2_5")
model.addConstr(x[0] + x[1] <= 1 - x[5], "exclusion_1_2_6")
model.addConstr(x[0] + x[1] <= 1 - x[6], "exclusion_1_2_7")


# If any location from 19-22 is chosen, locations 24, 26, and 27 cannot be chosen
model.addConstr(quicksum(x[i] for i in range(18, 22)) <= 1 - x[23], "exclusion_19_22_24")
model.addConstr(quicksum(x[i] for i in range(18, 22)) <= 1 - x[25], "exclusion_19_22_26")
model.addConstr(quicksum(x[i] for i in range(18, 22)) <= 1 - x[26], "exclusion_19_22_27")


# If any location from 1-5 is chosen, at least one odd site from 21-27 must be chosen
model.addConstr(quicksum(x[i] for i in range(5)) <= quicksum(x[i] for i in range(20, 27, 2)), "odd_site_requirement_1_5")

model.addConstr(quicksum(x[i] for i in range(14)) == quicksum(x[i] for i in range(14, 27)), "equal_locations_1_14_15_27")
model.addConstr(quicksum(y[i] for i in range(9)) == quicksum(y[i] for i in range(18, 27)), "equal_units_1_9_19_27")

<gurobi.Constr *Awaiting Model Update*>

In [9]:
# Optimize the model
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 93 rows, 54 columns and 291 nonzeros
Model fingerprint: 0x242ab608
Variable types: 27 continuous, 27 integer (27 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [2e-01, 3e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+06]
Presolve removed 45 rows and 9 columns
Presolve time: 0.00s
Presolved: 48 rows, 45 columns, 192 nonzeros
Variable types: 18 continuous, 27 integer (27 binary)
Found heuristic solution: objective 1.085550e+07

Root relaxation: objective 6.765500e+06, 14 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    6765500.0000 6765500.00  0.00%    

In [10]:
# Check if an optimal solution is found
if model.status == GRB.OPTIMAL:
    print("Optimal solution found!")
    # Print the selected locations and the number of units transported
    selected_locations = [i + 1 for i in range(27) if x[i].X > 0.5]
    print("Selected locations:", selected_locations)
    for i in selected_locations:
        print(f"Units transported to location {i}: {y[i-1].X}")
else:
    print("No optimal solution found.")

Optimal solution found!
Selected locations: [11, 13, 15, 16]
Units transported to location 11: 175000.0
Units transported to location 13: 175000.0
Units transported to location 15: 175000.0
Units transported to location 16: 175000.0
